In [3]:
import pandas as pd
import numpy as np
import itertools
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error

# Configuration and File Paths
file_paths = {
    'sig_info': 'GSE70138_Broad_LINCS_sig_info_2017-03-06.txt',
    'pert_info': 'GSE70138_Broad_LINCS_pert_info.txt',
    'gene_info': 'GSE70138_Broad_LINCS_gene_info_2017-03-06.txt',
    'gctx_data': 'GSE70138_A375_subset_expression.h5'
}

try:
    sig_info_df = pd.read_csv(file_paths['sig_info'], sep='\t')
    pert_info_df = pd.read_csv(file_paths['pert_info'], sep='\t')
    gene_info_df = pd.read_csv(file_paths['gene_info'], sep='\t')
    print("All metadata files loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading a metadata file: {e}")
    print("Please ensure all .txt files are in the correct directory.")
    exit("Exiting due to missing metadata files.")

# Parameters for the probabilistic factorial design model
K_ORDER_INTERACTION = 2 # Pairwise
NUM_GENES_TO_MODEL = 10
MAX_DRUGS_TO_MODEL = 120
np.random.seed(42)

# 1. Load the subset gene expression data (from HDF5)
expression_matrix = None
try:
    print(f"Attempting to load subset data from {file_paths['gctx_data']}...")
    expression_matrix = pd.read_hdf(file_paths['gctx_data'], key='expression_data')
    print(f"Gene expression matrix loaded successfully: {expression_matrix.shape} " \
          f"(rows: genes, columns: sig_ids)")
except FileNotFoundError:
    print(f"Subset HDF5 file not found at {file_paths['gctx_data']}.")
    print("Please ensure the generated HDF5 file is in the correct directory.")
    expression_matrix = None
except Exception as e:
    print(f"An unexpected error occurred while loading subset HDF5: {e}")
    print("Consider checking file integrity or memory availability.")
    expression_matrix = None

if expression_matrix is None:
    exit("Exiting because subset expression data could not be loaded.")

# 2. Filter for A375 cell line and relevant treatments
print("\n--- Filtering Data for A375 Cell Line and Treatments ---")

a375_signatures = sig_info_df[sig_info_df['cell_id'] == 'A375'].copy()
print(f"Initial A375 signatures found: {len(a375_signatures)}")

active_drug_signatures = a375_signatures[
    (a375_signatures['pert_type'] == 'trt_cp') &
    (a375_signatures['pert_iname'] != 'DMSO')
].copy()

print(f"A375 signatures with active compound treatments (excluding DMSO): " \
      f"{len(active_drug_signatures)}")

all_unique_drugs = active_drug_signatures['pert_iname'].unique()
print(f"Total unique drug treatments identified before sampling: {len(all_unique_drugs)}")

# 3. Apply drug limitation
unique_drugs = []
if MAX_DRUGS_TO_MODEL is not None and len(all_unique_drugs) > MAX_DRUGS_TO_MODEL:
    print(f"Limiting model to {MAX_DRUGS_TO_MODEL} randomly selected drugs.")
    unique_drugs = np.random.choice(all_unique_drugs, MAX_DRUGS_TO_MODEL, replace=False).tolist()
else:
    print(f"Using all {len(all_unique_drugs)} identified unique drug treatments.")
    unique_drugs = all_unique_drugs.tolist()

p_num_treatments = len(unique_drugs) 

print(f"\nModeling with {p_num_treatments} unique drug treatments.")
# print("Unique drugs selected:", unique_drugs)

# Create drug_to_idx and idx_to_drug mappings -> convert drug names to numerical indices for Boolean treatment vectors (x_m)
drug_to_idx = {drug: i for i, drug in enumerate(unique_drugs)}
idx_to_drug = {i: drug for i, drug in enumerate(unique_drugs)}

print("Drug to index mapping created.")


# 4. Construct treatment assignment vectors (x_m) for each A375 signature
print("\n--- Constructing Treatment Assignment Vectors (x_m) ---")

treatment_vectors_xm = {}
active_drug_signatures_for_model = active_drug_signatures[
    active_drug_signatures['pert_iname'].isin(unique_drugs) | # Direct single drug
    active_drug_signatures['pert_id'].apply(lambda x: any(drug in unique_drugs for
                                                           drug in str(x).split('|'))) # Combinations
].copy()

valid_sig_ids_for_model = expression_matrix.columns.intersection(
    active_drug_signatures_for_model['sig_id']
).tolist()
print(f"Using {len(valid_sig_ids_for_model)} samples (signatures) that have selected " \
      f"drugs and expression data.")


for sig_id in valid_sig_ids_for_model:
    row = active_drug_signatures_for_model[active_drug_signatures_for_model['sig_id'] == sig_id].iloc[0]
    xm = np.full(p_num_treatments, -1)

    pert_ids_for_sig = str(row['pert_id']).split('|')

    actual_drugs_applied = []
    for p_id in pert_ids_for_sig:
        if p_id in pert_info_df['pert_id'].values:
            pert_row = pert_info_df[pert_info_df['pert_id'] == p_id].iloc[0]
            pert_type = pert_row['pert_type']
            pert_iname = pert_row['pert_iname']

            if pert_type == 'trt_cp' and pert_iname != 'DMSO' and pert_iname in unique_drugs:
                actual_drugs_applied.append(pert_iname)
        elif p_id == 'DMSO':
            pass
    
    for drug_name in actual_drugs_applied:
        if drug_name in drug_to_idx:
            xm[drug_to_idx[drug_name]] = 1
    
    treatment_vectors_xm[sig_id] = xm

print(f"Generated x_m vectors for {len(treatment_vectors_xm)} active drug signatures.")


# 5. Construct Fourier Basis Functions and Design Matrix (X_design_df)
print("\n--- Constructing Fourier Design Matrix ---")

def get_fourier_term(x_vec, S_indices):
    """
    Calculates the Fourier basis term phi_S(x) = product_{i in S} x_i.
    x_vec must be in {-1, 1} format.
    """
    if not S_indices:
        return 1.0
    
    prod = 1.0
    for idx in S_indices:
        prod *= x_vec[idx]
    return prod

all_fourier_subsets = []
all_fourier_subsets.append(tuple()) # Add the empty set for the intercept term
for order in range(1, K_ORDER_INTERACTION + 1):
    for subset_indices in itertools.combinations(range(p_num_treatments), order):
        all_fourier_subsets.append(subset_indices)

print(f"Total number of Fourier basis terms (features) for p={p_num_treatments}, " \
      f"k={K_ORDER_INTERACTION}: {len(all_fourier_subsets)}")

X_rows = []
for sig_id in valid_sig_ids_for_model:
    x_m = treatment_vectors_xm[sig_id]
    row_x = []
    for S_indices in all_fourier_subsets:
        row_x.append(get_fourier_term(x_m, S_indices))
    X_rows.append(row_x)

X_design_np = np.array(X_rows)

feature_column_names = []
for S_indices in all_fourier_subsets:
    if not S_indices:
        feature_name = "Intercept"
    elif len(S_indices) == 1:
        feature_name = f"{idx_to_drug[S_indices[0]]}"
    else:
        feature_name = "*".join([idx_to_drug[idx] for idx in S_indices])
    feature_column_names.append(feature_name)

X_design_df = pd.DataFrame(X_design_np, columns=feature_column_names, index=valid_sig_ids_for_model)
print(f"Design matrix X_design_df created with shape: {X_design_df.shape}")


# 6. Select target gene(s) and prepare outcome vector (y)
print("\n--- Preparing Outcome Variable(s) (y) ---")

gene_info_df['pr_gene_id_str'] = gene_info_df['pr_gene_id'].astype(str).str.strip()
all_gene_ids_in_expression = expression_matrix.index.intersection(
    gene_info_df['pr_gene_id_str']
)
print(f"Total genes with expression data and info: {len(all_gene_ids_in_expression)}")

genes_to_model_ids = []
if NUM_GENES_TO_MODEL is not None and len(all_gene_ids_in_expression) > NUM_GENES_TO_MODEL:
    print(f"Randomly selecting {NUM_GENES_TO_MODEL} genes to model.")
    genes_to_model_ids = np.random.choice(all_gene_ids_in_expression, NUM_GENES_TO_MODEL,
                                          replace=False).tolist()
else:
    print(f"Modeling all {len(all_gene_ids_in_expression)} available genes.")
    genes_to_model_ids = all_gene_ids_in_expression.tolist()

gene_id_to_symbol = gene_info_df.set_index('pr_gene_id_str')['pr_gene_symbol'].to_dict()

hardcoded_gene_symbols = [
    'NUP133', 'NFATC4', 'KIAA0196', 'PSMD10', 'NFE2L2',
    'ZNF274', 'CAMSAP2', 'RNH1', 'SPTAN1', 'BNIP3'
]
genes_to_model_ids = []
for symbol in hardcoded_gene_symbols:
    matching_ids = gene_info_df[gene_info_df['pr_gene_symbol'] == symbol]['pr_gene_id_str'].tolist()
    if matching_ids:
        genes_to_model_ids.append(matching_ids[0])
    else:
        print(f"Warning: Gene symbol '{symbol}' not found in gene_info_df.")
print(f"Modeling {len(genes_to_model_ids)} specific genes as per report results.")


# 7. Fit Polynomial Regression Model (Ridge Regression) for each selected gene
print("\n--- Fitting Regression Models for Selected Genes ---")

modeling_results = {}

for gene_id in genes_to_model_ids:
    gene_symbol = gene_id_to_symbol.get(str(gene_id).strip(), f"UNKNOWN_SYMBOL_{gene_id}")
    print(f"\nModeling gene: {gene_symbol} (ID: {gene_id})")

    try:
        y = expression_matrix.loc[gene_id, valid_sig_ids_for_model].values
        
        model = Ridge(alpha=1.0, fit_intercept=False)
        model.fit(X_design_df, y)
        y_pred = model.predict(X_design_df)

        r2 = r2_score(y, y_pred)
        mse = mean_squared_error(y, y_pred)

        coef_df = pd.DataFrame({
            "Feature": X_design_df.columns,
            "Estimated Coefficient": model.coef_
        }).sort_values(by="Estimated Coefficient", key=abs, ascending=False)

        modeling_results[gene_symbol] = {
            'R2': r2,
            'MSE': mse,
            'Coefficients': coef_df
        }
        print(f"  Model R-squared: {r2:.4f}, MSE: {mse:.4f}")

    except KeyError as e:
        print(f"  Skipping gene {gene_symbol} (ID: {gene_id}) due to missing data: {e}")
    except Exception as e:
        print(f"  An error occurred while modeling gene {gene_symbol} (ID: {gene_id}): {e}")


# 8. Display Overall Results
print("\n--- Overall Modeling Results ---")

if not modeling_results:
    print("No genes were successfully modeled. Please check data and parameters.")
else:
    for gene_symbol, results in modeling_results.items():
        print(f"\n--- Results for Gene: {gene_symbol} ---")
        print(f"  R-squared: {results['R2']:.4f}")
        print(f"  Mean Squared Error: {results['MSE']:.4f}")
        print("  Top Estimated Coefficients:")
        print(results['Coefficients'].head(10).to_string())

print("\n--- Project Complete ---")
print("You have successfully performed a probabilistic factorial design analysis.")
print("The coefficients above indicate the estimated main and interaction effects of drugs on gene expression across the modeled genes.")
print("Consider performing this analysis for other genes or refining the model parameters (e.g., k, Ridge alpha).")

--- Starting Bioinformatics Project: Probabilistic Factorial Design for Drug-Gene Analysis ---
All metadata files loaded successfully.
Attempting to load subset data from GSE70138_A375_subset_expression.h5...
Gene expression matrix loaded successfully: (978, 1000) (rows: genes, columns: sig_ids)

--- Filtering Data for A375 Cell Line and Treatments ---
Initial A375 signatures found: 12740
A375 signatures with active compound treatments (excluding DMSO): 12050
Total unique drug treatments identified before sampling: 1768
Limiting model to 120 randomly selected drugs.

Modeling with 120 unique drug treatments.
Drug to index mapping created.

--- Constructing Treatment Assignment Vectors (x_m) ---
Using 66 samples (signatures) that have selected drugs and expression data.
Generated x_m vectors for 66 active drug signatures.

--- Constructing Fourier Design Matrix ---
Total number of Fourier basis terms (features) for p=120, k=2: 7261
Design matrix X_design_df created with shape: (66, 7261